<a href="https://colab.research.google.com/github/vbonato/cnnTestBench/blob/main/cnnTestBench.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [96]:
import ipywidgets as widgets
from IPython.display import display, clear_output
import os

# ========== GLOBAL VARIABLES ==========
layer_slider = widgets.IntSlider(value=3, min=1, max=10, step=1, description='NLayers:', continuous_update=False)
layer_widgets = {}  # Will hold widgets per layer
generate_button = widgets.Button(description="Generate Code", button_style='success')

# ========== UTILITY FUNCTIONS ==========

def update_layer_output_and_next_input(layer_idx):
    """Update output size E of layer_idx, and for next layer update its C and H (if exists)."""
    layer = layer_widgets[layer_idx]
    # Get H, C for this layer
    H = layer["H"].value
    C = layer["C"].value
    R = layer["R"].value
    S = layer["S"].value
    P = (R - 1) // 2
    E = ((H + 2 * P - R) // S) + 1
    if E < 1:
        E = 1
    layer_widgets[layer_idx]["E_val"] = E

    next_idx = layer_idx + 1
    if next_idx in layer_widgets:
        nl = layer_widgets[next_idx]
        # Update next layer's H and C
        nl["H"].value = E
        # next layer's C = this layer's M
        nl["C"].value = layer["M"].value

def on_param_change(change):
    # When any parameter changes in layer i, recompute for that and subsequent layers
    for i in range(1, layer_slider.value + 1):
        layer = layer_widgets[i]
        if change['owner'] in [layer["R"], layer["S"], layer["H"], layer["M"]]:
            update_layer_output_and_next_input(i)
            for j in range(i + 1, layer_slider.value + 1):
                update_layer_output_and_next_input(j)
            display_widgets()
            break

def attach_observers():
    for i in range(1, layer_slider.value + 1):
        layer = layer_widgets[i]
        layer["R"].observe(on_param_change, names='value')
        layer["S"].observe(on_param_change, names='value')
        layer["M"].observe(on_param_change, names='value')
        if i == 1:
            layer["H"].observe(on_param_change, names='value')
            layer["C"].observe(on_param_change, names='value')

def create_layer_widgets(num_layers):
    global layer_widgets
    layer_widgets = {}

    for i in range(1, num_layers + 1):
        if i == 1:
            # First layer: editable C and H
            widgets_dict = {
                "C": widgets.BoundedIntText(value=3, min=1, max=1024, description=f'C{i}:'),
                "M": widgets.BoundedIntText(value=16, min=1, max=1024, description=f'M{i}:'),
                "H": widgets.BoundedIntText(value=28, min=1, max=1024, description=f'H{i}:'),
                "R": widgets.BoundedIntText(value=3, min=1, max=11, description=f'R{i}:'),
                "S": widgets.BoundedIntText(value=1, min=1, max=4, description=f'S{i}:'),
            }
        elif i < num_layers:
            # Intermediate convolution layers: C and H auto-updated, but editable M,R,S
            widgets_dict = {
                "C": widgets.BoundedIntText(value=16, min=1, max=1024, description=f'C{i}:', disabled=True),
                "M": widgets.BoundedIntText(value=16, min=1, max=1024, description=f'M{i}:'),
                "H": widgets.BoundedIntText(value=28, min=1, max=1024, description=f'H{i}:', disabled=True),
                "R": widgets.BoundedIntText(value=3, min=1, max=11, description=f'R{i}:'),
                "S": widgets.BoundedIntText(value=1, min=1, max=4, description=f'S{i}:'),
            }
        else:
            # Last layer: fully connected layer
            # Here, we disable C, H, R, S; only M is editable (number of output units)
            widgets_dict = {
                "C": widgets.BoundedIntText(value=16, min=1, max=1024, description=f'C{i}:', disabled=True),
                "M": widgets.BoundedIntText(value=1, min=1, max=1024, description=f'M{i}:'),
                "H": widgets.BoundedIntText(value=28, min=1, max=1024, description=f'H{i}:', disabled=True),
                "R": widgets.BoundedIntText(value=1, min=1, max=1, description=f'R{i}:', disabled=True),
                "S": widgets.BoundedIntText(value=1, min=1, max=1, description=f'S{i}:', disabled=True),
            }
        layer_widgets[i] = widgets_dict

    # Now update all layers in sequence
    for i in range(1, num_layers + 1):
        update_layer_output_and_next_input(i)

    attach_observers()

# ========== DISPLAY ==========

def display_widgets():
    clear_output(wait=True)
    display(layer_slider)
    for i in range(1, layer_slider.value + 1):
        wlist = [
            layer_widgets[i]["C"],
            layer_widgets[i]["M"],
            layer_widgets[i]["H"],
            layer_widgets[i]["R"],
            layer_widgets[i]["S"],
        ]
        display(widgets.HBox(wlist))
    display(generate_button)

def on_layer_slider_change(change):
    if change['name'] == 'value' and change['type'] == 'change':
        create_layer_widgets(change['new'])
        display_widgets()
        print_help()

layer_slider.observe(on_layer_slider_change)

# ========== PARAM COLLECTION & VALIDATION ==========

def collect_layer_params(num_layers, layer_widgets):
    layers = {}
    for i in range(1, num_layers + 1):
        C = layer_widgets[i]["C"].value
        M = layer_widgets[i]["M"].value
        H = layer_widgets[i]["H"].value
        R = layer_widgets[i]["R"].value
        S = layer_widgets[i]["S"].value
        layers[i] = {"C": C, "M": M, "H": H, "R": R, "S": S}
    return layers

def compute_output_sizes(layers):
    output_sizes = {}
    num_layers = len(layers)
    for i in range(1, num_layers + 1):
        H = layers[i]["H"]
        R = layers[i]["R"]
        S = layers[i]["S"]
        P = (R - 1) // 2
        E = ((H + 2*P - R) // S) + 1
        if E < 1:
            raise ValueError(f"Layer {i} output size E={E} invalid")
        output_sizes[i] = {"E": E, "P": P}
        if i < num_layers:
            next_H = layers[i+1]["H"]
            next_C = layers[i+1]["C"]
            if E != next_H:
                raise ValueError(f"Layer {i} output size E={E} does not match next H={next_H}")
            if layers[i]["M"] != next_C:
                raise ValueError(f"Layer {i} M={layers[i]['M']} does not match next C={next_C}")
    return output_sizes

# ========== CODE GENERATION ==========

def generate_convh_code(num_layers, layers, outputs):
    guard = f'CONV{num_layers}_H'
    code = f"#ifndef {guard}\n#define {guard}\n\n#include <cstddef>\ntypedef unsigned type_t;\n\n"
    for i in range(1, num_layers + 1):
        code += f"const size_t C{i} = {layers[i]['C']};\n"
        code += f"const size_t M{i} = {layers[i]['M']};\n"
        code += f"const size_t H{i} = {layers[i]['H']};\n"
        code += f"const size_t R{i} = {layers[i]['R']};\n"
        code += f"const size_t S{i} = {layers[i]['S']};\n"
        code += f"const size_t E{i} = {outputs[i]['E']};\n"
        code += f"const size_t P{i} = {outputs[i]['P']};\n\n"
    for i in range(1, num_layers):
        code += f"void conv{i}(type_t I{i}[C{i}*H{i}*H{i}], type_t W{i}[M{i}*C{i}*R{i}*R{i}], type_t O{i}[M{i}*E{i}*E{i}]);\n"
    # FC layer declaration: flatten to 1D input
    last = num_layers
    prev = last - 1
    flatten_size = layers[prev]['M'] * outputs[prev]['E'] * outputs[prev]['E']
    code += f"void fc_layer(type_t input[{flatten_size}], type_t W{last}[M{last}][{flatten_size}], type_t output[{layers[last]['M']}]);\n"
    code += "void cnn(type_t *input"
    for i in range(1, num_layers + 1):
        code += f", type_t *W{i}"
    code += ", type_t *output);\n"
    code += "#endif\n"
    return code

def generate_conv_code(num_layers, layers, outputs):
    code = f'#include "conv{num_layers}.h"\n\ntype_t relu(type_t x) {{ return (x > 0) ? x : 0; }}\n\n'

    for i in range(1, num_layers):
        code += f"void conv{i}(type_t I{i}[C{i}*H{i}*H{i}], type_t W{i}[M{i}*C{i}*R{i}*R{i}], type_t O{i}[M{i}*E{i}*E{i}]) {{\n"
        code += f"    for(int m = 0; m < M{i}; m++) for(int c = 0; c < C{i}; c++) for(int y = 0; y < E{i}; y++) for(int x = 0; x < E{i}; x++)\n"
        code += f"    for(int k = 0; k < R{i}; k++) for(int l = 0; l < R{i}; l++) {{\n"
        code += f"        int h1 = y * S{i} - P{i} + k;\n"
        code += f"        int h2 = x * S{i} - P{i} + l;\n"
        code += f"        type_t val = (h1 < 0 || h1 >= (int)H{i} || h2 < 0 || h2 >= (int)H{i}) ? 0 : I{i}[h2 + (h1 + (c * H{i})) * H{i}];\n"
        code += f"        O{i}[x + (y + (m * E{i})) * E{i}] += val * W{i}[l + (k + (c + (m * C{i})) * R{i}) * R{i}];\n"
        code += f"    }}\n}}\n\n"

    last = num_layers
    prev = last - 1
    flatten_size = layers[prev]['M'] * outputs[prev]['E'] * outputs[prev]['E']

    code += f"void fc_layer(type_t input[{flatten_size}], type_t W{last}[M{last}][{flatten_size}], type_t output[{layers[last]['M']}]) {{\n"
    code += f"    for(int m = 0; m < M{last}; m++) {{\n"
    code += f"        output[m] = 0;\n"
    code += f"        for(int i = 0; i < {flatten_size}; i++) output[m] += input[i] * W{last}[m][i];\n"
    code += f"    }}\n}}\n\n"

    code += f"void cnn(type_t *input"
    for i in range(1, num_layers + 1):
        code += f", type_t *W{i}"
    code += ", type_t *output) {\n#pragma HLS DATAFLOW\n"

    for i in range(1, num_layers):
        code += f"    static type_t O{i}[M{i}*E{i}*E{i}];\n"

    for i in range(1, num_layers):
        if i == 1:
            code += f"    conv1(input, W1, O1);\n"
        else:
            code += f"    conv{i}(O{i-1}, W{i}, O{i});\n"
        code += f"    for(int j = 0; j < M{i} * E{i} * E{i}; j++) O{i}[j] = relu(O{i}[j]);\n"

    code += f"    type_t flat[{flatten_size}];\n"
    code += f"    for(int m = 0; m < M{prev}; m++) for(int y = 0; y < E{prev}; y++) for(int x = 0; x < E{prev}; x++) {{\n"
    code += f"        int idx = x + (y + m*E{prev}) * E{prev};\n"
    code += f"        flat[idx] = O{prev}[idx];\n"
    code += f"    }}\n"
    code += f"    fc_layer(flat, W{last}, output);\n"
    code += "}\n"

    return code


# ========== BUTTON CALLBACK ==========

def on_generate_clicked(b):
    num_layers = layer_slider.value
    layers = collect_layer_params(num_layers, layer_widgets)
    outputs = compute_output_sizes(layers)
    convh = generate_convh_code(num_layers, layers, outputs)
    convc = generate_conv_code(num_layers, layers, outputs)
    folder = "./generated"
    os.makedirs(folder, exist_ok=True)
    with open(f"{folder}/conv{num_layers}.h", "w") as f:
        f.write(convh)
    with open(f"{folder}/conv{num_layers}.c", "w") as f:
        f.write(convc)
    print(f"Generated conv{num_layers}.h and conv{num_layers}.c in {folder}")

generate_button.on_click(on_generate_clicked)

# ========== HELP TEXT ==========

def print_help():
    print("Set parameters for each convolutional layer:")
    print("- C: Number of input channels (e.g., 3 for RGB images)")
    print("     * For Layer 1, C1 is editable.")
    print("     * For layers > 1, C is auto-set to previous layer's M (output channels) and disabled.")
    print("- M: Number of output channels (filters)")
    print("- H: Input spatial height/width")
    print("     * For Layer 1, H1 is editable.")
    print("     * For layers > 1, H is auto-computed from previous layer output size and disabled.")
    print("- R: Filter size (kernel size, e.g., 3 for 3x3)")
    print("- S: Stride (step size for convolution)")
    print("Press 'Generate Code' to produce .h and .c files for the CNN.")

# ========== INITIAL SETUP ==========

create_layer_widgets(layer_slider.value)
display_widgets()
print_help()


IntSlider(value=3, continuous_update=False, description='NLayers:', max=10, min=1)

Button(button_style='success', description='Generate Code', style=ButtonStyle())

In [64]:
# CNN Code Simulation for n Layers Wrapper

import os
import ipywidgets as widgets
from IPython.display import display, clear_output
import subprocess
from git import Repo
import re

destination_directory = "./repo"

# Read constants from convX.h and extract inline comments
def read_constants(file_path):
    constants = {}
    comments = {}
    current_comment = ""
    with open(file_path, 'r') as file:
        for line in file:
            line = line.strip()
            if line.startswith("//"):
                current_comment = line
            elif line.startswith("const size_t"):
                parts = line.split()
                if len(parts) >= 5:
                    const_name = parts[2]
                    try:
                        const_val = int(parts[4].strip(";"))
                        constants[const_name] = const_val
                        comments[const_name] = current_comment
                        current_comment = ""
                    except ValueError:
                        continue
    return constants, comments

# Write updated constants back to convX.h
def modify_conv_constants(file_path, updated_constants):
    with open(file_path, 'r') as file:
        lines = file.readlines()

    new_lines = []
    for line in lines:
        if line.strip().startswith("const size_t"):
            parts = line.strip().split()
            if len(parts) >= 5:
                name = parts[2]
                if name in updated_constants:
                    new_line = f"const size_t {name} = {updated_constants[name]};\n"
                    new_lines.append(new_line)
                    continue
        new_lines.append(line)

    with open(file_path, 'w') as file:
        file.writelines(new_lines)
    print(f"✅ Updated constants in {os.path.basename(file_path)}")

# Run shell command
def run_command(cwd, command):
    result = subprocess.run(command, cwd=cwd, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)
    if result.returncode == 0:
        print("✅ Command executed successfully:\n", result.stdout)
    else:
        print("❌ Error during execution:\n", result.stderr)

# Show constants as widgets for editing
def display_constant_widgets(constants, comments):
    widgets_list = []
    for name in sorted(constants.keys()):
        label = widgets.HTML(value=f"<b>{comments.get(name, '')}</b>")
        field = widgets.IntText(value=constants[name], description=name, layout=widgets.Layout(width='160px'))
        widgets_list.append(label)
        widgets_list.append(field)

    save_button = widgets.Button(description="Update code & make", button_style='primary')

    def on_save_click(_):
        for w in widgets_list:
            if isinstance(w, widgets.IntText):
                constants[w.description] = w.value
        modify_conv_constants(configure_path, constants)
        conv_num = re.search(r'(\d+)', file_dropdown.value).group(1)
        makefile_name = f"Makefile{conv_num}"
        run_command(os.path.join(destination_directory, "DynamicCNN"), ['make', '-f', makefile_name])

    save_button.on_click(on_save_click)
    display(widgets.VBox(widgets_list + [save_button]))

# Scan repo directory for convX.h files
def get_conv_h_files(directory):
    if not os.path.exists(directory):
        return []
    return sorted([f for f in os.listdir(directory) if re.match(r"^conv\d+\.h$", f)])

# Load selected convX.h and update the display
def update_paths_and_constants(_=None):
    global configure_path, constants, comments
    configure_path = os.path.join(destination_directory, "DynamicCNN", file_dropdown.value)
    constants, comments = read_constants(configure_path)
    clear_output(wait=True)
    display(file_dropdown)
    display_constant_widgets(constants, comments)
    display(run_button, reset_button)

# Run the generated CNN binary
def run_code_click(_):
    conv_num = re.search(r'(\d+)', file_dropdown.value).group(1)
    exec_name = f"./conv{conv_num}"
    exec_path = os.path.join(destination_directory, "DynamicCNN", exec_name)

    # Just to be extra sure: check if the file exists
    if not os.path.exists(exec_path):
        print(f"❌ Executable not found: {exec_path}")
        return

    run_command(os.path.dirname(exec_path), [exec_name])


# Clear all UI and redisplay
def reset_output(_):
    clear_output(wait=True)
    display(file_dropdown)
    display_constant_widgets(constants, comments)
    display(run_button, reset_button)

# UI setup
conv_h_files = get_conv_h_files(os.path.join(destination_directory, "DynamicCNN"))

if not conv_h_files:
    print("❌ No convX.h files found in ./repo/DynamicCNN. Please generate code first.")
else:
    file_dropdown = widgets.Dropdown(
        options=conv_h_files,
        value=conv_h_files[0],
        description='Layers:',
        layout=widgets.Layout(width='200px')
    )
    file_dropdown.observe(update_paths_and_constants, names='value')

    configure_path = os.path.join(destination_directory, "DynamicCNN", file_dropdown.value)
    constants, comments = read_constants(configure_path)

    run_button = widgets.Button(description="Run as x86", button_style='success')
    reset_button = widgets.Button(description="Clear Display", button_style='warning')

    run_button.on_click(run_code_click)
    reset_button.on_click(reset_output)

    display(file_dropdown)
    display_constant_widgets(constants, comments)
    display(run_button, reset_button)


Dropdown(description='Layers:', index=1, layout=Layout(width='200px'), options=('conv2.h', 'conv3.h', 'conv4.h…

Button(button_style='success', description='Run as x86', style=ButtonStyle())

Button(button_style='warning', description='Clear Display', style=ButtonStyle())



---



---



---


below old for one layer - version working



---



---



---




In [ ]:
#-------------------------------------------
# CNN Code generator for i number of layers
#-------------------------------------------

import ipywidgets as widgets
from IPython.display import display, clear_output

import os

# Function to generate the makefile with the specified number of layers
def generate_makefile_code(num_layers):
    makefile_lines = []

    # Target for the main executable
    makefile_lines.append(f'conv{num_layers}: conv{num_layers}.o conv_tb{num_layers}.o')
    makefile_lines.append(f'\tclang conv{num_layers}.o conv_tb{num_layers}.o -o conv{num_layers}\n')

    makefile_lines.append(f'conv_tb{num_layers}.o: conv_tb{num_layers}.cpp conv_tb{num_layers}.h conv{num_layers}.h')
    makefile_lines.append(f'\tclang -c conv_tb{num_layers}.cpp -o conv_tb{num_layers}.o\n')

    makefile_lines.append(f'conv{num_layers}.o: conv{num_layers}.cpp conv{num_layers}.h')
    makefile_lines.append(f'\tclang -c conv{num_layers}.cpp -o conv{num_layers}.o\n')

    # Clean target
    makefile_lines.append('.PHONY: clean')
    makefile_lines.append('clean:')
    makefile_lines.append('\trm -rf ' + f'\tconv{num_layers}.o conv_tb{num_layers}.o conv{num_layers} ' + ' '.join([f'output{j}.bin' for j in range(1, num_layers + 1)]) + '\n')

    # Join code lines to form the final output
    generated_makefile = '\n'.join(makefile_lines)
    return generated_makefile

# Function to generate the testbench code with the specified number of layers
def generate_testbench_code(num_layers):
    code_lines = []

    # Add header includes
    code_lines.append('#include <cstdio>')
    code_lines.append('#include <cstdlib>')
    code_lines.append(f'\n#include "conv_tb{num_layers}.h"')  # Updated to use num_layers
    code_lines.append(f'#include "conv{num_layers}.h"\n')  # Updated to use num_layers

    # Start of the main function
    code_lines.append('int main(void) {')

    # Declare input, weights, and output arrays
    for i in range(num_layers):
        code_lines.append(f'    type_t *I{i + 1} = (type_t *) malloc(C{i + 1} * H{i + 1} * H{i + 1} * sizeof(type_t));')
        code_lines.append(f'    type_t *W{i + 1} = (type_t *) malloc(M{i + 1} * C{i + 1} * R{i + 1} * R{i + 1} * sizeof(type_t));')
        code_lines.append(f'    type_t *OL{i + 1} = (type_t *) calloc(M{i + 1} * E{i + 1} * E{i + 1}, sizeof(type_t));')

    # Random initialization of inputs
    code_lines.append('\n    srand(1);')
    for i in range(num_layers):
        code_lines.append(f'    for(unsigned j = 0; j < C{i + 1} * H{i + 1} * H{i + 1}; j++)')
        code_lines.append(f'        I{i + 1}[j] = rand() % RANDROOF;')

    # Random initialization of weights
    for i in range(num_layers):
        code_lines.append(f'    for(unsigned j = 0; j < M{i + 1} * C{i + 1} * R{i + 1} * R{i + 1}; j++)')
        code_lines.append(f'        W{i + 1}[j] = rand() % RANDROOF;')

    # Convolution function calls
    code_lines.append('\n    // Perform convolutions')
    for i in range(num_layers):
        code_lines.append(f'    conv{i + 1}(I{i + 1}, W{i + 1}, OL{i + 1});')

    # Print output
    code_lines.append('\n    // Output results')
    code_lines.append(f'    for(int j = 0; j < M{num_layers} * E{num_layers} * E{num_layers}; j++) {{')
    code_lines.append(f'        printf("%x ", OL{num_layers}[j]);')
    code_lines.append('    }\n')

    # Write output to files
    code_lines.append('    // Write output to files')
    for i in range(num_layers):
        code_lines.append(f'    FILE *opf{i + 1} = fopen("output{i + 1}.bin", "wb");')
        code_lines.append(f'    fwrite(OL{i + 1}, sizeof(type_t), M{i + 1} * E{i + 1} * E{i + 1}, opf{i + 1});')
        code_lines.append(f'    fclose(opf{i + 1});')

    # Free allocated memory
    code_lines.append('\n    // Free allocated memory')
    for i in range(num_layers):
        code_lines.append(f'    if(I{i + 1}) free(I{i + 1});')
        code_lines.append(f'    if(W{i + 1}) free(W{i + 1});')
        code_lines.append(f'    if(OL{i + 1}) free(OL{i + 1});')

    # End of main function
    code_lines.append('\n    return EXIT_SUCCESS;')
    code_lines.append('}')

    # Join code lines to form the final output
    generated_code = '\n'.join(code_lines)
    return generated_code

# Function to generate the testbench header with the specified number of layers
def generate_testbench_header_code(num_layers):
    code_lines = []

    # Add header includes
    code_lines.append('#ifndef CONV_TB_H')
    code_lines.append('#define CONV_TB_H')
    code_lines.append('#define RANDROOF 256')
    code_lines.append('#endif')

     # Join code lines to form the final output
    generated_code = '\n'.join(code_lines)
    return generated_code


# Function to generate the convolution implementation code with the specified number of layers
def generate_conv_code(num_layers):
    code = "#include \"conv{}.h\"\n\n".format(num_layers)  # Header file for the specific layer count

    # Generate convolution functions for each layer
    for i in range(1, num_layers + 1):
        code += "void conv{}(type_t I{}[C{} * H{} * H{}], type_t W{}[M{} * C{} * R{} * R{}], type_t O{}[M{} * E{} * E{}]) {{\n".format(
            i, i, i, i, i, i, i, i, i, i, i, i, i, i)
        code += "    // LOF\n"
        code += "    for(int m = 0; m < M{}; m++) {{\n".format(i)
        code += "        // LIF\n"
        code += "        for(int c = 0; c < C{}; c++) {{\n".format(i)
        code += "            // LSY\n"
        code += "            for(int y = 0; y < E{}; y++) {{\n".format(i)
        code += "                // LSX\n"
        code += "                for(int x = 0; x < E{}; x++) {{\n".format(i)
        code += "                    // LFY\n"
        code += "                    for(int k = 0; k < R{}; k++) {{\n".format(i)
        code += "                        // LFX\n"
        code += "                        for(int l = 0; l < R{}; l++) {{\n".format(i)
        code += "                            int h1 = y * S{} - P{} + k;\n".format(i, i)
        code += "                            int h2 = x * S{} - P{} + l;\n".format(i, i)
        code += "                            type_t i = (h1 < 0 || h1 >= H{} || h2 < 0 || h2 >= H{}) ? 0 : I{}[h2 + (h1 + (c * H{})) * H{}];\n".format(
            i, i, i, i, i)
        code += "                            O{}[x + (y + (m * E{})) * E{}] += i * W{}[l + (k + (c + (m * C{})) * R{}) * R{}];\n".format(
            i, i, i, i, i, i, i)
        code += "                        }\n"  # End LFX
        code += "                    }\n"  # End LFY
        code += "                }\n"  # End LSX
        code += "            }\n"  # End LSY
        code += "        }\n"  # End LIF
        code += "    }\n"  # End LOF
        code += "}\n\n"  # End of function

    return code

# Function to generate the convolution header code with the specified number of layers
def generate_convh_code(num_layers):
    code = "#ifndef CONV_H\n#define CONV_H\n\n#include <cstddef>\n\ntypedef unsigned type_t;\n\n"

    # Generate code for each section of the configuration for each layer
    code += "// Number of input feature maps (N. channels in)\n"
    for i in range(1, num_layers + 1):
        code += "const size_t C{} = 256;\n".format(i)

    code += "\n// Number of output feature maps (N. channels out)\n"
    for i in range(1, num_layers + 1):
        code += "const size_t M{} = 256;\n".format(i)

    code += "\n// Input feature map size (H x H)\n"
    for i in range(1, num_layers + 1):
        code += "const size_t H{} = 6;\n".format(i)

    code += "\n// Convolution kernel size (R x R)\n"
    for i in range(1, num_layers + 1):
        code += "const size_t R{} = 5;\n".format(i)

    code += "\n// Convolution kernel stride\n"
    for i in range(1, num_layers + 1):
        code += "const size_t S{} = 1;\n".format(i)

    code += "\n// Output feature map size (E x E)\n"
    for i in range(1, num_layers + 1):
        code += "const size_t E{} = 6;\n".format(i)

    code += "\n// Input feature map size with padding (F x F)\n"
    for i in range(1, num_layers + 1):
        code += "const size_t F{} = ((E{} * S{} + R{} - 1) < H{}) ? H{} : (E{} * S{} + R{} - 1);\n".format(i, i, i, i, i, i, i, i, i)

    code += "\n// Padding size\n"
    for i in range(1, num_layers + 1):
        code += "const size_t P{} = (F{} - H{}) / 2;\n".format(i, i, i)

    # Add convolution function declarations
    code += "\n"
    for i in range(1, num_layers + 1):
        code += "void conv{}(type_t I{}[C{} * H{} * H{}], type_t W{}[M{} * C{} * R{} * R{}], type_t O{}[M{} * E{} * E{}]);\n".format(
            i, i, i, i, i, i, i, i, i, i, i, i, i, i)

    code += "\n#endif"

    return code


# Create a slider for user input
layer_slider = widgets.IntSlider(
    value=3,
    min=1,
    max=10,  # Adjust as needed
    step=1,
    description='NLayers:',
    continuous_update=False
)

# Create a button to generate code
generate_button = widgets.Button(
    description='Generate code',
    button_style='success'
)

# Set up the button's click event
def on_button_click(b):
    clear_output(wait=True)
    display(layer_slider, generate_button)

    # Define the folder path
    folder_path = './repo/padlogicNLayers'

    # Create the folder if it doesn't exist
    os.makedirs(folder_path, exist_ok=True)

    # Define the file path
    file_path = f'{folder_path}/Makefile{layer_slider.value}'

    generated_makefile = generate_makefile_code(layer_slider.value)
    # Save to MakefileX
    with open(f'{folder_path}/Makefile{layer_slider.value}', 'w') as makefile_file:
        makefile_file.write(generated_makefile)
    print(f'Makefile{layer_slider.value} has been created and saved.')

    generate_testbench = generate_testbench_code(layer_slider.value)
    # Save to testbench
    with open(f'{folder_path}/conv_tb{layer_slider.value}.cpp', 'w') as makefile_file:
        makefile_file.write(generate_testbench)
    print(f'conv_tb{layer_slider.value}.cpp has been created and saved.')

    generate_testbench_header = generate_testbench_header_code(layer_slider.value)
    # Save to MakefileX
    with open(f'{folder_path}/conv_tb{layer_slider.value}.h', 'w') as makefile_file:
        makefile_file.write(generate_testbench_header)
    print(f'conv_tb{layer_slider.value}.h has been created and saved.')

    generate_convh = generate_convh_code(layer_slider.value)
    # Save to conv header
    with open(f'{folder_path}/conv{layer_slider.value}.h', 'w') as makefile_file:
        makefile_file.write(generate_convh)
    print(f'conv{layer_slider.value}.h has been created and saved.')

    generate_conv = generate_conv_code(layer_slider.value)
    # Save to conv cpp
    with open(f'{folder_path}/conv{layer_slider.value}.cpp', 'w') as makefile_file:
        makefile_file.write(generate_conv)
    print(f'conv{layer_slider.value}.cpp has been created and saved.')

generate_button.on_click(on_button_click)

# Display the slider and button
display(layer_slider, generate_button)


In [ ]:
import os
import ipywidgets as widgets
from IPython.display import display, clear_output
import subprocess
from git import Repo
import re  # Import regular expression module

destination_directory = "./repo"

# # Function to clone a GitHub repository if not exist
# def clone_repo(github_url, dest_dir):
#     # Check if the destination directory exists
#     if os.path.exists(dest_dir):
#         # Check if the directory is a Git repository
#         if os.path.exists(os.path.join(dest_dir, ".git")):
#             print(f"Repository already cloned in {dest_dir}.")
#             return  # Exit the function if the repository is already cloned
#         else:
#             print(f"Directory {dest_dir} exists but is not a Git repository.")
#             # Optionally, you could choose to remove the existing directory or proceed differently
#             return

#     # Create the directory if it doesn't exist and clone the repository
#     os.makedirs(dest_dir)
#     Repo.clone_from(github_url, dest_dir)
#     print(f"Cloned repository to {dest_dir}")

# Clone the GitHub repository
#github_url = "https://github.com/vbonato/paunajacaTestBench.git"
#clone_repo(github_url, destination_directory)


# Function to read constants and comments from .h file
def read_constants(file_path):
    constants = {}
    comments = {}
    current_comment = ""

    with open(file_path, 'r') as file:
        lines = file.readlines()
        for line in lines:
            line = line.strip()
            # Collect comment lines
            if line.startswith("//"):
                current_comment = line  # Update to last comment line
            elif line.startswith("const size_t"):
                parts = line.split()
                constant_name = parts[2]
                try:
                    constant_value = int(parts[4].strip(";"))
                    constants[constant_name] = constant_value
                    # Store the last comment line
                    comments[constant_name] = current_comment.strip()  # Last line of comment
                    current_comment = ""  # Reset comment for the next constant
                except ValueError:
                    # Skip lines that don't contain simple integer values
                    continue
    return constants, comments

# Function to modify .h file with new values for constants
def modify_conv_constants(file_path, constants):
    with open(file_path, 'r') as file:
        lines = file.readlines()

    new_lines = []
    for line in lines:
        if line.startswith("const size_t"):
            parts = line.split()
            constant_name = parts[2]
            if constant_name in constants:
                new_lines.append(f"const size_t {constant_name} = {constants[constant_name]};\n")
            else:
                new_lines.append(line)
        else:
            new_lines.append(line)

    with open(file_path, 'w') as file:
        file.writelines(new_lines)
    print(f"Updated constants in {file_path}")

# Function to run a command (Makefile or executable)
def run_command(destination_directory, command):
    result = subprocess.run(command, cwd=destination_directory, stdout=subprocess.PIPE, stderr=subprocess.PIPE, universal_newlines=True)
    if result.returncode == 0:
        print("Command executed successfully!")
        print(result.stdout)
    else:
        print("Error executing command:")
        print(result.stderr)

# Function to display widgets for constant modification along with comments
def display_constant_widgets(constants, comments):
    widgets_list = []
    for name, value in constants.items():
        comment = comments.get(name, "")  # Get the associated last-line comment (if any)
        comment_label = widgets.HTML(value=f"<b>{comment}</b>")  # Display comment as HTML label
        constant_widget = widgets.IntText(value=value, description=name, layout=widgets.Layout(width='150px'))

        widgets_list.append(comment_label)
        widgets_list.append(constant_widget)

    # Create a button to save the changes
    save_button = widgets.Button(description="Update code & make")

    def on_save_button_click(b):
        # Update constants with new values from widgets
        for widget in widgets_list:
            if isinstance(widget, widgets.IntText):
                constants[widget.description] = widget.value
        modify_conv_constants(configure_path, constants)

        # Extract the number from the selected convX.h and select the corresponding Makefile
        conv_number = re.search(r'(\d+)', file_dropdown.value).group(1)  # Extract the number from convX
        selected_makefile = f"Makefile{conv_number}"  # Build the Makefile name like Makefile1
        run_command(os.path.join(destination_directory, "padlogicNLayers"), ['make', '-f', selected_makefile])

    save_button.on_click(on_save_button_click)

    # Display the widgets and button
    display(widgets.VBox(widgets_list + [save_button]))

# Function to display buttons
def display_buttons():
    display(run_button, reset_button)


# Scan the directory for convX.h files and populate the dropdown options
def get_conv_h_files(directory):
    # List all files in the directory and filter for files matching convX.h pattern
    return [f for f in os.listdir(directory) if re.match(r"^conv\d+\.h$", f)]

# Update paths and constants based on the selected .h file
def update_paths_and_constants():
    global configure_path, constants, comments
    configure_path = os.path.join(destination_directory, "padlogicNLayers", file_dropdown.value)
    constants, comments = read_constants(configure_path)
    clear_output(wait=True)  # Clear output to refresh displayed widgets
    display(file_dropdown)  # Re-display the dropdown
    display_constant_widgets(constants, comments)  # Display updated widgets
    display_buttons()  # Re-display buttons

# Get convX.h files from the 'padlogicNLayers' directory
conv_h_files = get_conv_h_files(os.path.join(destination_directory, "padlogicNLayers"))

# Dropdown to choose between convX.h files found in the repository
file_dropdown = widgets.Dropdown(
    options=conv_h_files,
    value=conv_h_files[0] if conv_h_files else '',  # Set default value if available
    description='Layers:',
    layout=widgets.Layout(width='180px')
)
file_dropdown.observe(lambda change: update_paths_and_constants(), names='value')

# Initial configuration path setup
configure_path = os.path.join(destination_directory, "padlogicNLayers", file_dropdown.value)
constants, comments = read_constants(configure_path)

# Create a button to run the code
run_button = widgets.Button(description="Run as x86")
reset_button = widgets.Button(description="Clear Display")

# Function to run the code
def run_code_click(b):
    # Extract the conv number from the dropdown value and select the corresponding executable
    conv_number = re.search(r'(\d+)', file_dropdown.value).group(1)  # Extract the number from convX
    executable = f'./conv{conv_number}'  # Build executable name like ./conv1
    run_command(os.path.join(destination_directory, "padlogicNLayers"), [executable])

# Function to reset the output
def reset_output(b):
    clear_output(wait=True)  # Clear all output
    display(file_dropdown)  # Display the dropdown to choose between available convX.h files
    display_constant_widgets(constants, comments)  # Re-display constant widgets and comments
    display_buttons()  # Re-display run/reset buttons

# Set button clicks
run_button.on_click(run_code_click)
reset_button.on_click(reset_output)

# Initial display
display(file_dropdown)  # Display the dropdown to choose between available convX.h files
display_constant_widgets(constants, comments)
display_buttons()


In [ ]:
# BKP
# CNN Code Generator with n Layers Wrapper -

import ipywidgets as widgets
from IPython.display import display, clear_output
import os

def generate_makefile_code(num_layers):
    makefile_lines = [
        f'conv{num_layers}: conv{num_layers}.o conv_tb{num_layers}.o',
        f'\tclang conv{num_layers}.o conv_tb{num_layers}.o -o conv{num_layers}\n',

        f'conv_tb{num_layers}.o: conv_tb{num_layers}.cpp conv_tb{num_layers}.h conv{num_layers}.h',
        f'\tclang -c conv_tb{num_layers}.cpp -o conv_tb{num_layers}.o\n',

        f'conv{num_layers}.o: conv{num_layers}.cpp conv{num_layers}.h',
        f'\tclang -c conv{num_layers}.cpp -o conv{num_layers}.o\n',

        '.PHONY: clean',
        'clean:',
        '\trm -rf ' + f'conv{num_layers}.o conv_tb{num_layers}.o conv{num_layers} ' +
        ' '.join([f'output{j}.bin' for j in range(1, num_layers + 1)]) + '\n'
    ]
    return '\n'.join(makefile_lines)

def generate_testbench_code(num_layers):
    code_lines = [
        '#include <cstdio>',
        '#include <cstdlib>',
        f'#include "conv_tb{num_layers}.h"',
        f'#include "conv{num_layers}.h"\n',
        'int main(void) {'
    ]

    # Buffers
    code_lines.append(f'    type_t *I1 = (type_t *) malloc(C1 * H1 * H1 * sizeof(type_t));')
    for i in range(1, num_layers + 1):
        code_lines.append(f'    type_t *W{i} = (type_t *) malloc(M{i} * C{i} * R{i} * R{i} * sizeof(type_t));')
    code_lines.append(f'    type_t *O{num_layers} = (type_t *) calloc(M{num_layers} * E{num_layers} * E{num_layers}, sizeof(type_t));')

    # Init inputs
    code_lines.append('\n    srand(1);')
    code_lines.append('    for(unsigned j = 0; j < C1 * H1 * H1; j++)')
    code_lines.append('        I1[j] = rand() % RANDROOF;')
    for i in range(1, num_layers + 1):
        code_lines.append(f'    for(unsigned j = 0; j < M{i} * C{i} * R{i} * R{i}; j++)')
        code_lines.append(f'        W{i}[j] = rand() % RANDROOF;')

    # CNN Call
    code_lines.append('\n    // Perform CNN inference with pipelined conv layers')
    cnn_call = '    cnn(I1'
    for i in range(1, num_layers + 1):
        cnn_call += f', W{i}'
    cnn_call += f', O{num_layers});'
    code_lines.append(cnn_call)

    # Print output
    code_lines.append(f'\n    for(int j = 0; j < M{num_layers} * E{num_layers} * E{num_layers}; j++) {{')
    code_lines.append(f'        printf("%x ", O{num_layers}[j]);')
    code_lines.append('    }\n')

    # Output to file
    code_lines.append(f'    FILE *opf = fopen("output{num_layers}.bin", "wb");')
    code_lines.append(f'    fwrite(O{num_layers}, sizeof(type_t), M{num_layers} * E{num_layers} * E{num_layers}, opf);')
    code_lines.append('    fclose(opf);')

    # Free
    code_lines.append('    if(I1) free(I1);')
    for i in range(1, num_layers + 1):
        code_lines.append(f'    if(W{i}) free(W{i});')
    code_lines.append(f'    if(O{num_layers}) free(O{num_layers});')

    code_lines.append('\n    return EXIT_SUCCESS;\n}')
    return '\n'.join(code_lines)

def generate_testbench_header_code(num_layers):
    return '#ifndef CONV_TB_H\n#define CONV_TB_H\n#define RANDROOF 256\n#endif'

def generate_conv_code(num_layers):
    code = f'#include "conv{num_layers}.h"\n\n'

    # Add ReLU function
    code += "type_t relu(type_t x) {\n"
    code += "    return (x > 0) ? x : 0;\n"
    code += "}\n\n"

    # Generate convolution layer functions
    for i in range(1, num_layers):
        code += f"void conv{i}(type_t I{i}[C{i} * H{i} * H{i}], type_t W{i}[M{i} * C{i} * R{i} * R{i}], type_t O{i}[M{i} * E{i} * E{i}]) {{\n"
        code += f"    for(int m = 0; m < M{i}; m++) {{\n"
        code += f"        for(int c = 0; c < C{i}; c++) {{\n"
        code += f"            for(int y = 0; y < E{i}; y++) {{\n"
        code += f"                for(int x = 0; x < E{i}; x++) {{\n"
        code += f"                    for(int k = 0; k < R{i}; k++) {{\n"
        code += f"                        for(int l = 0; l < R{i}; l++) {{\n"
        code += f"                            int h1 = y * S{i} - P{i} + k;\n"
        code += f"                            int h2 = x * S{i} - P{i} + l;\n"
        code += f"                            type_t val = (h1 < 0 || h1 >= H{i} || h2 < 0 || h2 >= H{i}) ? 0 : I{i}[h2 + (h1 + (c * H{i})) * H{i}];\n"
        code += f"                            O{i}[x + (y + (m * E{i})) * E{i}] += val * W{i}[l + (k + (c + (m * C{i})) * R{i}) * R{i}];\n"
        code += f"                        }}\n" * 3
        code += f"    }}\n" * 3
        code += f"}}\n\n"

    # Final FC layer as the last layer
    code += f"void fc_layer(type_t input[C{num_layers} * H{num_layers} * H{num_layers}], type_t W{num_layers}[C{num_layers} * H{num_layers} * H{num_layers}], type_t output[1]) {{\n"
    code += f"    output[0] = 0;\n"
    code += f"    for (int i = 0; i < C{num_layers} * H{num_layers} * H{num_layers}; i++) {{\n"
    code += f"        output[0] += input[i] * W{num_layers}[i];\n"
    code += f"    }}\n"
    code += f"}}\n\n"

    # Wrapper CNN function
    code += "void cnn(type_t *input"
    for i in range(1, num_layers + 1):
        code += f", type_t *W{i}"
    code += ", type_t *output) {\n"
    code += "#pragma HLS DATAFLOW\n"

    for i in range(1, num_layers):
        code += f"    static type_t O{i}[M{i} * E{i} * E{i}];\n"

    code += "\n"

    for i in range(1, num_layers + 1):
        if i == 1:
            code += f"    conv1(input, W1, O1);\n"
            code += f"    for (int j = 0; j < M1 * E1 * E1; j++) O1[j] = relu(O1[j]);\n"
        elif i < num_layers:
            code += f"    conv{i}(O{i-1}, W{i}, O{i});\n"
            code += f"    for (int j = 0; j < M{i} * E{i} * E{i}; j++) O{i}[j] = relu(O{i}[j]);\n"
        else:
            code += f"    fc_layer(O{num_layers - 1}, W{num_layers}, output);\n"

    code += "}\n"
    return code


# def generate_conv_code(num_layers):
#     code = f'#include "conv{num_layers}.h"\n\n'
#     for i in range(1, num_layers + 1):
#         code += f"void conv{i}(type_t I{i}[C{i} * H{i} * H{i}], type_t W{i}[M{i} * C{i} * R{i} * R{i}], type_t O{i}[M{i} * E{i} * E{i}]) {{\n"
#         code += f"    for(int m = 0; m < M{i}; m++) {{\n"
#         code += f"        for(int c = 0; c < C{i}; c++) {{\n"
#         code += f"            for(int y = 0; y < E{i}; y++) {{\n"
#         code += f"                for(int x = 0; x < E{i}; x++) {{\n"
#         code += f"                    for(int k = 0; k < R{i}; k++) {{\n"
#         code += f"                        for(int l = 0; l < R{i}; l++) {{\n"
#         code += f"                            int h1 = y * S{i} - P{i} + k;\n"
#         code += f"                            int h2 = x * S{i} - P{i} + l;\n"
#         code += f"                            type_t val = (h1 < 0 || h1 >= H{i} || h2 < 0 || h2 >= H{i}) ? 0 : I{i}[h2 + (h1 + (c * H{i})) * H{i}];\n"
#         code += f"                            O{i}[x + (y + (m * E{i})) * E{i}] += val * W{i}[l + (k + (c + (m * C{i})) * R{i}) * R{i}];\n"
#         code += f"                        }}\n" * 3
#         code += f"    }}\n" * 3
#         code += f"}}\n\n"

#     # Add cnn() wrapper
#     code += "void cnn(type_t *input"
#     for i in range(1, num_layers + 1):
#         code += f", type_t *W{i}"
#     code += f", type_t *output) {{\n#pragma HLS DATAFLOW\n"
#     for i in range(1, num_layers):
#         code += f"    static type_t O{i}[M{i} * E{i} * E{i}];\n"
#     code += "\n"
#     if num_layers == 1:
#         code += f"    conv1(input, W1, output);\n"
#     else:
#         for i in range(1, num_layers + 1):
#             if i == 1:
#                 code += f"    conv1(input, W1, O1);\n"
#             elif i == num_layers:
#                 code += f"    conv{num_layers}(O{num_layers-1}, W{num_layers}, output);\n"
#             else:
#                 code += f"    conv{i}(O{i-1}, W{i}, O{i});\n"
#     code += "}\n"
#     return code


def generate_convh_code(num_layers):
    code = "#ifndef CONV_H\n#define CONV_H\n\n#include <cstddef>\ntypedef unsigned type_t;\n\n"
    # for section, label in [("C", "channels in"), ("M", "channels out"), ("H", "input height"),
    #                        ("R", "kernel size"), ("S", "stride"), ("E", "output size")]:
    #     code += f"// {label}\n"
    #     for i in range(1, num_layers + 1):
    #         code += f"const size_t {section}{i} = 256;\n" if section in "CM" else f"const size_t {section}{i} = 6;\n"
    #     code += "\n"

    # Generate code for each section of the configuration for each layer
    code += "// Number of input feature maps (N. channels in)\n"
    for i in range(1, num_layers + 1):
        code += "const size_t C{} = 256;\n".format(i)

    code += "\n// Number of output feature maps (N. channels out)\n"
    for i in range(1, num_layers + 1):
        code += "const size_t M{} = 256;\n".format(i)

    code += "\n// Input feature map size (H x H)\n"
    for i in range(1, num_layers + 1):
        code += "const size_t H{} = 6;\n".format(i)

    code += "\n// Convolution kernel size (R x R)\n"
    for i in range(1, num_layers + 1):
        code += "const size_t R{} = 5;\n".format(i)

    code += "\n// Convolution kernel stride\n"
    for i in range(1, num_layers + 1):
        code += "const size_t S{} = 1;\n".format(i)

    code += "\n// Output feature map size (E x E)\n"
    for i in range(1, num_layers + 1):
        code += "const size_t E{} = 6;\n".format(i)

    for i in range(1, num_layers + 1):
        code += f"const size_t F{i} = ((E{i} * S{i} + R{i} - 1) < H{i}) ? H{i} : (E{i} * S{i} + R{i} - 1);\n"
        code += f"const size_t P{i} = (F{i} - H{i}) / 2;\n"
    code += "\n"
    for i in range(1, num_layers + 1):
        code += f"void conv{i}(type_t I{i}[C{i} * H{i} * H{i}], type_t W{i}[M{i} * C{i} * R{i} * R{i}], type_t O{i}[M{i} * E{i} * E{i}]);\n"

    # code += "\nvoid cnn(type_t *input"
    # for i in range(1, num_layers + 1):
    #     code += f", type_t *W{i}"
    # code += ", type_t *output);\n"

    code += "\nvoid cnn(type_t *input"
    for i in range(1, num_layers + 1):
      code += f", type_t *W{i}"
    code += ", type_t *output);\n"

    code += f"void fc_layer(type_t input[C{num_layers} * H{num_layers} * H{num_layers}], type_t W{num_layers}[C{num_layers} * H{num_layers} * H{num_layers}], type_t output[1]);\n"
    code += "\n#endif"
    return code

# GUI
layer_slider = widgets.IntSlider(value=3, min=1, max=10, step=1, description='NLayers:', continuous_update=False)
generate_button = widgets.Button(description='Generate code', button_style='success')

def on_button_click(b):
    clear_output(wait=True)
    display(layer_slider, generate_button)
    folder = './repo/padlogicNLayers'
    os.makedirs(folder, exist_ok=True)
    i = layer_slider.value

    with open(f'{folder}/Makefile{i}', 'w') as f: f.write(generate_makefile_code(i))
    with open(f'{folder}/conv_tb{i}.cpp', 'w') as f: f.write(generate_testbench_code(i))
    with open(f'{folder}/conv_tb{i}.h', 'w') as f: f.write(generate_testbench_header_code(i))
    with open(f'{folder}/conv{i}.h', 'w') as f: f.write(generate_convh_code(i))
    with open(f'{folder}/conv{i}.cpp', 'w') as f: f.write(generate_conv_code(i))

    print(f'✅ All files for {i} layers generated in {folder}')

generate_button.on_click(on_button_click)
display(layer_slider, generate_button)


IntSlider(value=3, continuous_update=False, description='NLayers:', max=10, min=1)

Button(button_style='success', description='Generate code', style=ButtonStyle())

✅ All files for 3 layers generated in ./repo/padlogicNLayers
